# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [67]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [68]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'Heart_Disease_AutoML'

experiment=Experiment(ws, experiment_name)
print("Experiment created")

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136845
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-136845
Experiment created


In [69]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating a compute cluster
# vm_size = "Standard_D2_V2"
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Exploring the Dataset

In [70]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

df = pd.read_csv(url_path)

In [71]:
ds_dataframe = ds.to_pandas_dataframe()
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [72]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.head()

Shape of dataset before split: (302, 21)


,age,resting_BP,cholesterol,max_heart_rate,st_depression,sex_1,chest_pain_type_1,chest_pain_type_2,chest_pain_type_3,fasting_blood_sugar_1,rest_ECG_1,rest_ECG_2,exercise_induced_angina_1,st_slope_1,st_slope_2,num_major_vessels_1.0,num_major_vessels_2.0,num_major_vessels_3.0,thalassemia_2.0,thalassemia_3.0
0,63,145,233,150,2.3,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
1,37,130,250,187,3.5,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
2,41,130,204,172,1.4,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
3,56,120,236,178,0.8,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0
4,57,120,354,163,0.6,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0


In [73]:
import os
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory 'Data' created...")
except OSError:
    print("Directory 'Data' cannot be created...")

Dicrectory 'Data' created...


In [75]:
from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state = 42)

train_df = train_x.copy(deep=True)
train_df['target'] = train_y

# joining series y column to the test data
test_df = test_x.copy(deep=True)
test_df['target'] = test_y

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

print(train_df.shape)
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv", index=False)
test_df.to_csv("./Data/test_data.csv", index=False)

(241, 20) (241,) (61, 20) (61,)
(241, 21)
(61, 21)


In [76]:
# upload the Data directory to the datastore
deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="./Data", target_path="./Data", overwrite=True)

Uploading an estimated of 2 files
Uploading ./Data/test_data.csv
Uploaded ./Data/test_data.csv, 1 files out of an estimated total of 2
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_84ae731783934199997369ef2a829d8c

In [77]:
from azureml.core import Dataset

train_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/train_data.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/test_data.csv'))])

In [78]:
train_data.to_pandas_dataframe()

,age,resting_BP,cholesterol,max_heart_rate,st_depression,sex_1,chest_pain_type_1,chest_pain_type_2,chest_pain_type_3,fasting_blood_sugar_1,...,rest_ECG_2,exercise_induced_angina_1,st_slope_1,st_slope_2,num_major_vessels_1.0,num_major_vessels_2.0,num_major_vessels_3.0,thalassemia_2.0,thalassemia_3.0,target
0,42,120,295,162,0.0,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1
1,68,180,274,150,1.6,1,0,1,0,1,...,0,1,1,0,0,0,0,0,1,0
2,67,125,254,163,0.2,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
3,55,135,250,161,1.4,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,1
4,64,140,335,158,0.0,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,41,110,172,158,0.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
237,51,94,227,154,0.0,1,0,1,0,0,...,0,1,0,1,1,0,0,0,1,1
238,69,160,234,131,0.1,1,0,0,1,1,...,0,0,1,0,1,0,0,1,0,1
239,61,134,234,145,2.6,1,0,0,1,0,...,0,0,1,0,0,1,0,1,0,0


In [79]:
train_data

{
  "source": [
    "('workspaceblobstore', 'Data/train_data.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [18]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_config = AutoMLConfig (
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = train_data,
    label_column_name = "target",
    n_cross_validations = 5,
    enable_onnx_compatible_models = True,
    compute_target = cpu_cluster)

print("AutoML config created.")

AutoML config created.


In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_630c4303-243c-48f9-9972-ce1471f6309c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)

In [ ]:
print(fitted_model)

In [ ]:
best_run.get_tags()

In [ ]:
import joblib
from azureml.core.model import Model

#TODO: Save the best model

description = "Best Model from AutoML run of the Cleveland Foundation Clinic Heart Disease Dataset"
automl_model = remote_run.register_model(model_name='heart_disease_autoML_best_model.pkl', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service